In [2]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from scipy.special import softmax
# !pip install googletrans==4.0.0-rc1
from googletrans import Translator

In [3]:
vk_data = pd.read_excel("VK_full.xlsx")
rbc_data = pd.read_excel("rbc_full_crypto_articles_to_2024-05-24.xlsx")

# Предобработаем данные

### Для начала уберем NaN

In [4]:
vk_data.isnull().sum()


Уникальный идентификатор записи       0
Дата и время публикации               0
Текст записи                       1886
Количество комментариев               0
Информация о лайках                   0
Количество просмотров                 0
dtype: int64

In [5]:

vk_data = vk_data.dropna(subset=['Текст записи'])


In [6]:
vk_data.isnull().sum()

Уникальный идентификатор записи    0
Дата и время публикации            0
Текст записи                       0
Количество комментариев            0
Информация о лайках                0
Количество просмотров              0
dtype: int64

In [7]:
rbc_data.isnull().sum()

Unnamed: 0        0
0                 2
1                 0
2                 0
3                 0
4                 0
5             12375
6                 0
7                 0
8                 0
9                 0
10                0
11             2359
12            12375
13                0
14            12375
15                0
16                0
17                0
dtype: int64

In [8]:
rbc_data = rbc_data.iloc [: , 2:]
rbc_data = rbc_data[1:]
rbc_data.reset_index(drop= True , inplace= True)
rbc_data = rbc_data.drop(index = 4795)
rbc_data.reset_index(drop= True , inplace= True)

In [9]:
rbc_data = rbc_data.rename(columns={1: 'id', 2: 'project', 3: 'project_nick', 4: 'type', 5: 'category', 6: 'title',7 : 'body', 8: 'publish_date', 9: 'publish_date_t', 10: 'fronturl', 11: 'picture', 12: 'badge', 13: 'pay_option', 14: 'data', 15: '_score', 16: 'overview', 17: 'text'})

In [10]:
rbc_data['publish_date'] = pd.to_datetime(rbc_data['publish_date']).dt.date

In [11]:
vk_data['Дата и время публикации'] = pd.to_datetime(vk_data['Дата и время публикации']).dt.date

In [12]:
rbc_data.isnull().sum()

id                    0
project               0
project_nick          0
type                  0
category          12375
title                 0
body                  0
publish_date          0
publish_date_t        0
fronturl              0
picture            2359
badge             12375
pay_option            0
data              12375
_score                0
overview              0
text                  0
dtype: int64

# проверка видеокарты

In [13]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3070


##### мне вывело
##### "There are 1 GPU(s) available.
##### We will use the GPU: NVIDIA GeForce RTX 3070"
##### Если у вас не подключена видеокарта, то скачайте torch по полной ссылке на сайте https://pytorch.org/

# Модель BERT и классификация

Модель 1

In [14]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("r1char9/rubert-base-cased-russian-sentiment")
# model = AutoModelForSequenceClassification.from_pretrained("r1char9/rubert-base-cased-russian-sentiment")

Модель 2

0: POSITIVE
1: NEGATIVE

In [18]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-mokoron')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-mokoron', return_dict=True)


In [19]:
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    #predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

In [100]:
rbk_postive = []
rbk_negative = []
for i in range(len(rbc_data['title'])):
    tensorr = predict(rbc_data['title'][i])
    rbk_postive.append(tensorr[0][0].item())
    rbk_negative.append(tensorr[0][1].item())


In [28]:
vk_data.reset_index(drop= True , inplace= True)

In [32]:
vk_postive = []
vk_negative = []
for i in range(len(vk_data['Текст записи'])):
    tensorr = predict(vk_data['Текст записи'][i])
    vk_postive.append(tensorr[0][0].item())
    vk_negative.append(tensorr[0][1].item())

In [33]:
len(vk_data)

12512

In [35]:
len(vk_negative)

12512

In [37]:
# vk_data['vk_posotive_rubert'] = vk_postive
# vk_data['vk_negative_rubert'] = vk_negative

In [39]:
columns_to_save_VK = ['Дата и время публикации', 'vk_posotive_rubert', 'vk_negative_rubert']
vk_data.to_csv('vk_RuBeart_text_Sentiment.csv', columns=columns_to_save_VK, index=False)

In [ ]:
vk_data = vk_data.rename(columns={'Текст записи': 'Post text'})

In [ ]:
vk_data

In [108]:
# rbc_data['rbk_positive'] = rbk_postive
# rbc_data['rbk_negative'] = rbk_negative

In [112]:
columns_to_save = ['publish_date', 'rbk_positive', 'rbk_negative']

In [113]:
rbc_data.to_csv('rbk_RuBeart_Titles_Sentiment.csv', columns=columns_to_save, index=False)

# Этот берт оказался не очень (ухудшил RMSE, попробую обученного крипто Берта ElKulako/cryptobert, но он работает для английских текстов (нужно перевести))

Скачиваю криптоберта

In [55]:
# Load model directly 
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ElKulako/cryptobert")
model = AutoModelForSequenceClassification.from_pretrained("ElKulako/cryptobert")

Перевод текста

текст я перевожу в отдельном файле translator.py здесь я просто сохраняю и получаю csv файл


In [42]:
columns_to_save_for_VK_translator = ['Post text']

In [47]:
vk_data['Post text'].iloc[0]

'За всем происходящем в мире криптовалют следим в нашем тг \nhttps://t.me/+zx0CCbfwek41OGNi ↩'

In [48]:
vk_data.to_csv('vk_text.csv', columns=columns_to_save_for_VK_translator, index=True)

In [49]:
len(vk_data)

12512

In [135]:
# Импортируем CSV файл в DataFrame
trans_rbc_titles = pd.read_csv("translated_rbc.csv")

In [51]:
trans_vk_text = pd.read_csv("translated_vk_text.csv")

Итак вот наш переведенный текст

In [139]:
trans_rbc_titles['title']


0        The most famous cloud mining service confirmed...
1        No time to explain: everything about cryptocur...
2        You might have missed it: exchanges will add S...
3        A major exchange has returned to work after a ...
4        There will be no public ICO round. All details...
                               ...                        
12370    The US approved the launch of exchange-traded ...
12371    The main trends of the cryptocurrency market i...
12372    Experts called the Solana cryptocurrency suita...
12373    What will happen to the Ethereum rate after ET...
12374    Cryptocurrency volatility has decreased in ant...
Name: title, Length: 12375, dtype: object

In [69]:
ct = 0
for i in range(len(trans_vk_text['Post text'])):
    if len(str(trans_vk_text['Post text'][i])) > ct:
        ct = len(str(trans_vk_text['Post text'][i]))
        
print(ct)        

13271


In [76]:
(trans_vk_text['Post text'][11817]) 

'[club143763740|Топ 9 правил криптотрейдинга или как превратить 1000$ в 46000$ меньше чем за год]\n\nХочу сразу сказать, что это не моя история успеха, а одного успешного человека и трейдера, чье имя по понятным причинам я назвать не могу. Всем, кто когда-либо занимался трейдингом криптовалют, хоть раз да повезло, однако, чтобы достичь весомого результат большинству приходится пробовать разные стратегии, вступать в сообщества криптотрейдеров, отслеживать текущие тенденции и т.д. Некоторые результаты можно отнести к категории «повезло», в то время как общий успех достигается тяжелым трудом, в основе которого лежат фундаментальные принципы, хорошие привычки и опыт. \n \n💰💰💰9 правил прибыльного криптотрейдинга \nВ то время как рынки криптовалют чрезвычайно волатильны, и все инвесторы подвержены колебаниям цен, включая меня и вас и всех остальных успешных трейдеров, имея хорошие привычки в торговле можно смягчить потери и максимизировать прибыль. \n \nОбратите внимание, что ни один пункт и

In [77]:
for i in range(len(trans_vk_text['Post text'])):
    if len(str(trans_vk_text['Post text'][i])) > 1000:
        trans_vk_text = trans_vk_text.drop(i) 
   

In [78]:
len(trans_vk_text['Post text'])

9231

In [79]:
def predict(text):
    inputs = tokenizer(text, max_length=1001, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    #predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

In [85]:
trans_vk_text.reset_index(drop= True , inplace= True)

In [86]:
vk_postive_cryptobert = []
vk_negative_cryptobert = []
vk_neutral_cryptobert = []
for i in range(len(trans_vk_text)):
    tensorr = predict(trans_vk_text['Post text'][i])
    vk_postive_cryptobert.append(tensorr[0][2].item())
    vk_negative_cryptobert.append(tensorr[0][0].item())
    vk_neutral_cryptobert.append(tensorr[0][1].item())

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [93]:
len(vk_neutral_cryptobert)

4754

In [148]:
rbk_postive_cryptobert = []
rbk_negative_cryptobert = []
rbk_neutral_cryptobert = []

for i in range(len(trans_rbc_titles)):
    tensorr = predict(trans_rbc_titles['title'][i])
    rbk_postive_cryptobert.append(tensorr[0][2].item())
    rbk_negative_cryptobert.append(tensorr[0][0].item())
    rbk_neutral_cryptobert.append(tensorr[0][1].item())

"Bearish" : 0, "Neutral": 1, "Bullish": 2

In [149]:
tensorr

tensor([[0.0006, 0.6389, 0.3605]], grad_fn=<SoftmaxBackward0>)

In [151]:
rbc_data['rbk_postive_cryptobert'] = rbk_postive_cryptobert
rbc_data['rbk_negative_cryptobert'] = rbk_negative_cryptobert
rbc_data['rbk_neutral_cryptobert'] = rbk_neutral_cryptobert 

In [96]:
vk_data

,Уникальный идентификатор записи,Дата и время публикации,Post text,Количество комментариев,Информация о лайках,Количество просмотров,vk_posotive_rubert,vk_negative_rubert
0,125701,2022-10-19,За всем происходящем в мире криптовалют следим...,1,518,118742,0.527412,0.472588
1,156939,2024-05-24,"Центробанк РФ заявил, что активность россиян н...",4,12,677,0.479314,0.520686
2,156907,2024-05-23,"По мнению аналитика Bloomberg, SEC одобрит ETF...",9,14,3835,0.000120,0.999880
3,156897,2024-05-23,"AI сектор у всех на слуху, что купить? \n\n• S...",3,24,2757,0.671547,0.328453
4,156894,2024-05-23,"В России рассмотрели спор о возврате USDT, кот...",1,21,5663,0.550594,0.449406
...,...,...,...,...,...,...,...,...
12507,283,2017-04-12,money-clash.ru ⛔СКАМ! НЕ ВКЛАДЫВАТЬ.\n⛔⛔⛔Админ...,0,6,292,0.553791,0.446209
12508,88,2017-04-05,Каждый хорошо построенный дом начинался с план...,0,25,542,0.503072,0.496928
12509,87,2017-04-05,"Деньги для людей умных составляют средство, дл...",0,24,538,0.443439,0.556561
12510,80,2017-04-05,"Наш большой недостаток в том, что мы слишком б...",0,7,359,0.418640,0.581360


In [152]:
columns_to_save = ['publish_date', 'rbk_postive_cryptobert', 'rbk_negative_cryptobert', 'rbk_neutral_cryptobert']

In [153]:
rbc_data.to_csv('rbk_data_from_cryptobert.csv', columns=columns_to_save, index=False)

## Готово (получен датасет с криптоберта)